In [5]:
%load_ext autoreload
%autoreload 2
!which python
%env CUDA_VISIBLE_DEVICES=8

/home/guandao/tava/env/tava/bin/python
env: CUDA_VISIBLE_DEVICES=8


In [6]:
import numpy as np
import torch
import os.path as osp
import pickle
import meshplot as mp
import trimesh
import sys 
sys.path.insert(0, "/home/guandao/tava/")

In [26]:
aid = 3
humanrf_smpl_out_dir = "/home/guandao/tava/data/actorhq_dataset/Actor%02d/Sequence1/4x/tava/smpl" % aid
humanrf_smplx_out_dir = "/home/guandao/tava/data/actorhq_dataset/Actor%02d/Sequence1/4x/smplx_fitted_icp" % aid
# humanrf_smplx_out_dir = "/home/guandao/tava/data/humanrf_processed/smplx_fitted_icp"
!ls {humanrf_smpl_out_dir}

000060.obj  000307.pkl	000555.obj  000802.pkl	001050.obj  001297.pkl
000060.pkl  000308.obj	000555.pkl  000803.obj	001050.pkl  001298.obj
000061.obj  000308.pkl	000556.obj  000803.pkl	001051.obj  001298.pkl
000061.pkl  000309.obj	000556.pkl  000804.obj	001051.pkl  001299.obj
000062.obj  000309.pkl	000557.obj  000804.pkl	001052.obj  001299.pkl
000062.pkl  000310.obj	000557.pkl  000805.obj	001052.pkl  001300.obj
000063.obj  000310.pkl	000558.obj  000805.pkl	001053.obj  001300.pkl
000063.pkl  000311.obj	000558.pkl  000806.obj	001053.pkl  001301.obj
000064.obj  000311.pkl	000559.obj  000806.pkl	001054.obj  001301.pkl
000064.pkl  000312.obj	000559.pkl  000807.obj	001054.pkl  001302.obj
000065.obj  000312.pkl	000560.obj  000807.pkl	001055.obj  001302.pkl
000065.pkl  000313.obj	000560.pkl  000808.obj	001055.pkl  001303.obj
000066.obj  000313.pkl	000561.obj  000808.pkl	001056.obj  001303.pkl
000066.pkl  000314.obj	000561.pkl  000809.obj	001056.pkl  001304.obj
000067.obj  000314.pkl	000562.obj 

In [27]:
fid = 805
frame_id = "%06d" % fid
print("Frame ID:", frame_id)
!ls -lah {humanrf_smpl_out_dir}/{frame_id}*
!ls {humanrf_smplx_out_dir}/{frame_id}/*

Frame ID: 000805
-rw-rw-r-- 1 guandao guandao 427K Feb 23 11:50 /home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/tava/smpl/000805.obj
-rw-rw-r-- 1 guandao guandao 359K Feb 23 11:50 /home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/tava/smpl/000805.pkl
/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/smplx_fitted_icp/000805/detection.json
/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/smplx_fitted_icp/000805/smplx_icp.obj
/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/smplx_fitted_icp/000805/smplx_icp_param.pth
/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/smplx_fitted_icp/000805/smplx.obj
/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/smplx_fitted_icp/000805/smplx_param.pth


In [28]:
# Verify the smpl fitting is good!
smpl_mesh = trimesh.load_mesh(f"{humanrf_smpl_out_dir}/{frame_id}.obj")
smplx_mesh = trimesh.load_mesh(f"{humanrf_smplx_out_dir}/{frame_id}/smplx_icp.obj")
p = mp.plot(smpl_mesh.vertices, smpl_mesh.faces, return_plot=True)
p.add_points(smplx_mesh.vertices, shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0793140…

1

In [29]:
f"{humanrf_smpl_out_dir}/{frame_id}.pkl",  


('/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/tava/smpl/000805.pkl',)

In [30]:
# Plot the SMPL joints 
print(f"{humanrf_smpl_out_dir}/{frame_id}.pkl")
smpl_params = np.load(
    f"{humanrf_smpl_out_dir}/{frame_id}.pkl", allow_pickle=True)
for k, v in smpl_params.items():
    print(k, v.shape if v is not None else v)

/home/guandao/tava/data/actorhq_dataset/Actor03/Sequence1/4x/tava/smpl/000805.pkl
transl None
global_orient torch.Size([1, 1, 3, 3])
body_pose torch.Size([1, 23, 3, 3])
betas torch.Size([1, 10])
vertices torch.Size([1, 6890, 3])
joints torch.Size([1, 45, 3])
full_pose torch.Size([1, 24, 3, 3])
v_shaped None
v_posed torch.Size([1, 6890, 3])
transform_mat torch.Size([1, 24, 4, 4])
bone_transform_mat torch.Size([1, 24, 4, 4])
faces (13776, 3)


In [31]:
p = mp.plot(smpl_params["vertices"][0].detach().cpu().numpy())
# smpl_params["faces"], alpha=0.5)
p.add_points(smpl_params["joints"][0][:23].detach().cpu().numpy().reshape(-1, 3),
             shading={"point_size": 0.1, "point_color": "blue"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0793138…

1

In [32]:
root_offset = smpl_params["bone_transform_mat"][0, 0, :3, -1] # 23x4x4
print(root_offset)

root_joint = smpl_params["joints"][0, 0]
print(root_joint)

tensor([-0.0023, -0.2341,  0.0181], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([ 0.0919,  0.8707, -0.1873], device='cuda:0', grad_fn=<SelectBackward0>)


In [33]:
# Check whether the transform is the same
from tava.datasets import zju_parser
root_fp = "/home/guandao/tava/data/zju/"
parser = zju_parser.SubjectParser(
    subject_id=313, root_fp=root_fp)

for k, v in smpl_params.items():
    print(k, v.shape if v is not None else v)

tf_bones = smpl_params["bone_transform_mat"][0, 1:] # 23x4x4
global_transform = torch.eye(4)
global_transform[:3, -1] = smpl_params["transl"][0]
global_transform = global_transform[None, ...].repeat(tf_bones.shape[0], 1, 1)
tf_bones = torch.bmm(global_transform.to(tf_bones.device), tf_bones)
heads = tf_bones[:, :3, -1].cpu().detach().numpy() 
# print("HEADS", heads)
joints = smpl_params["joints"][0, :24].detach().cpu().numpy() # 24x3
tails = np.stack([
    joints[parser.JOINT_NAMES.index(parser.BONE_NAMES[i][1])]
    for i in range(len(parser.BONE_NAMES))
])
print("Joints", joints.shape)
print("Heads", heads.shape)
print("Tails", tails.shape)

p = mp.plot(smpl_params["vertices"][0].detach().cpu().numpy())
for i in range(len(parser.BONE_NAMES)):
    p.add_edges(np.concatenate([
            heads[i].reshape(1, 3), 
            tails[i].reshape(1, 3)
        ], axis=0).reshape(2, 3), 
        np.array([[0, 1]]),
        shading={"line_color": "red"})
    p.add_points(heads[i].reshape(1, 3),
                shading={"point_size": 0.1, "point_color": "blue"})
    p.add_points(tails[i].reshape(1, 3),
                shading={"point_size": 0.1, "point_color": "green"})

transl None
global_orient torch.Size([1, 1, 3, 3])
body_pose torch.Size([1, 23, 3, 3])
betas torch.Size([1, 10])
vertices torch.Size([1, 6890, 3])
joints torch.Size([1, 45, 3])
full_pose torch.Size([1, 24, 3, 3])
v_shaped None
v_posed torch.Size([1, 6890, 3])
transform_mat torch.Size([1, 24, 4, 4])
bone_transform_mat torch.Size([1, 24, 4, 4])
faces (13776, 3)


TypeError: 'NoneType' object is not subscriptable

In [11]:
# Load ZJU dataset to check the number of joints
zju_dir = "/home/guandao/tava/data/zju/CoreView_313"
zju_pose_data = torch.load(f"{zju_dir}/pose_data.pt")
for k, v in zju_pose_data.items():
    print(k, v.shape if v is not None else v)
idx = 10
p = mp.plot(zju_pose_data["verts"][idx].detach().cpu().numpy())
p.add_points(zju_pose_data["joints"][idx].detach().cpu().numpy(), 
             shading={"point_size": 0.1, "point_color": "blue"})

lbs_weights torch.Size([6890, 24])
rest_verts torch.Size([6890, 3])
rest_joints torch.Size([24, 3])
rest_tfs torch.Size([24, 4, 4])
rest_tfs_bone torch.Size([24, 4, 4])
verts torch.Size([1470, 6890, 3])
joints torch.Size([1470, 24, 3])
tfs torch.Size([1470, 24, 4, 4])
tf_bones torch.Size([1470, 24, 4, 4])
params torch.Size([1470, 78])


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.2522526…

1

In [43]:
# Load ZJU dataset to check the number of joints
from tava.datasets import zju_parser
root_fp = "/home/guandao/tava/data/zju/"
parser = zju_parser.SubjectParser(
    subject_id=313, root_fp=root_fp)
for k, v in zju_pose_data.items():
    print(k, v.shape if v is not None else v)
    
idx = 10
tf_bones = zju_pose_data["tf_bones"][idx][1:] # 23x3
heads = tf_bones[:, :3, -1]
joints = zju_pose_data["joints"][idx] # 24x3
tails = [
    joints[parser.JOINT_NAMES.index(parser.BONE_NAMES[i][1])] 
    for i in range(len(parser.BONE_NAMES))
]
p = mp.plot(zju_pose_data["verts"][idx].detach().cpu().numpy())
for i in range(len(parser.BONE_NAMES)):
    p.add_edges(np.stack([heads[i], tails[i]]).reshape(2, 3), 
                np.array([[0, 1]]),
                shading={"line_color": "red"});
    p.add_points(heads[i].detach().cpu().numpy().reshape(1, 3), 
                shading={"point_size": 0.1, "point_color": "blue"})
    p.add_points(tails[i].detach().cpu().numpy().reshape(1, 3), 
                shading={"point_size": 0.1, "point_color": "green"})

lbs_weights torch.Size([6890, 24])
rest_verts torch.Size([6890, 3])
rest_joints torch.Size([24, 3])
rest_tfs torch.Size([24, 4, 4])
rest_tfs_bone torch.Size([24, 4, 4])
verts torch.Size([1470, 6890, 3])
joints torch.Size([1470, 24, 3])
tfs torch.Size([1470, 24, 4, 4])
tf_bones torch.Size([1470, 24, 4, 4])
params torch.Size([1470, 78])


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.2522526…

In [13]:
# Load ZJU dataset to check the number of joints
from tava.datasets import zju_parser
root_fp = "/home/guandao/tava/data/zju/"
parser = zju_parser.SubjectParser(
    subject_id=313, root_fp=root_fp)
for k, v in zju_pose_data.items():
    print(k, v.shape if v is not None else v)
    
tf_bones = zju_pose_data["rest_tfs_bone"][1:] # 23x3
heads = tf_bones[:, :3, -1]
joints = zju_pose_data["rest_joints"] # 24x3
tails = [
    joints[parser.JOINT_NAMES.index(parser.BONE_NAMES[i][1])] 
    for i in range(len(parser.BONE_NAMES))
]
p = mp.plot(zju_pose_data["rest_verts"].detach().cpu().numpy())
for i in range(len(parser.BONE_NAMES)):
    p.add_edges(np.stack([heads[i], tails[i]]).reshape(2, 3), 
                np.array([[0, 1]]),
                shading={"line_color": "red"});
    p.add_points(heads[i].detach().cpu().numpy().reshape(1, 3), 
                shading={"point_size": 0.1, "point_color": "blue"})
    p.add_points(tails[i].detach().cpu().numpy().reshape(1, 3), 
                shading={"point_size": 0.1, "point_color": "green"})

lbs_weights torch.Size([6890, 24])
rest_verts torch.Size([6890, 3])
rest_joints torch.Size([24, 3])
rest_tfs torch.Size([24, 4, 4])
rest_tfs_bone torch.Size([24, 4, 4])
verts torch.Size([1470, 6890, 3])
joints torch.Size([1470, 24, 3])
tfs torch.Size([1470, 24, 4, 4])
tf_bones torch.Size([1470, 24, 4, 4])
params torch.Size([1470, 78])


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0006337…

In [14]:
zju_pose_data["rest_tfs_bone"]

tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.8577e-03],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00, -2.2603e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.9332e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.8577e-03],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00, -2.2603e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.9332e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.8577e-03],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00, -2.2603e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.9332e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.8577e-03],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00, -2.2603e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  2.9332e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

In [15]:
zju_pose_data["tf_bones"][10]

tensor([[[-6.4794e-01, -9.5663e-02, -7.5566e-01,  2.0389e-01],
         [-7.6037e-01,  1.3959e-01,  6.3431e-01, -2.0608e-01],
         [ 4.4801e-02,  9.8558e-01, -1.6318e-01,  9.1844e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[-6.4794e-01, -9.5663e-02, -7.5566e-01,  2.0389e-01],
         [-7.6037e-01,  1.3959e-01,  6.3431e-01, -2.0608e-01],
         [ 4.4801e-02,  9.8558e-01, -1.6318e-01,  9.1844e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[-6.4794e-01, -9.5663e-02, -7.5566e-01,  2.0389e-01],
         [-7.6037e-01,  1.3959e-01,  6.3431e-01, -2.0608e-01],
         [ 4.4801e-02,  9.8558e-01, -1.6318e-01,  9.1844e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[-6.4794e-01, -9.5663e-02, -7.5566e-01,  2.0389e-01],
         [-7.6037e-01,  1.3959e-01,  6.3431e-01, -2.0608e-01],
         [ 4.4801e-02,  9.8558e-01, -1.6318e-01,  9.1844e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 